In [57]:
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import re
import timeit
import datetime
import xgboost as xgb
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.graph_objs as go
import folium
import json
import utm
import scipy
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import joblib
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import folium
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import datetime
import scipy.stats as stats
import plotly.express as px
pd.set_option('mode.chained_assignment', None)

# libraries for import all file names
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression


## Analisis Anomalies

In [22]:
def calcularAnomaly(df, col, cont=0.005):
    #Crear una versio horaria del dataset
    df['timestamp'] = pd.to_datetime(df.index)
    df = df.set_index('timestamp').resample("H").mean().reset_index()
    
    #Usem el metode de Isolation Forest per trobar anomalies a les precipitacions
    output = df[[col, 'timestamp']]
    temp = df[[col, 'timestamp']]
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=cont,max_features=1.0)
    model.fit(temp[[col]].values)
    output['outliers']=pd.Series(model.predict(temp[[col]].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )
    output.rename(columns={col:'data'}, inplace=True)
    
    return output

In [23]:
# Carregar dades pluja
def loadDataPrecipitacions():
    precipitacions = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
    return precipitacions

In [24]:
# Carregar dades Alt Ter
def loadDataAltTer():
    altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
    santJoan = calcularAnomaly(altTer, "L17167-72-00001")
    massiesRoda = calcularAnomaly(altTer, "L08116-72-00002")
    
    return santJoan, massiesRoda

In [25]:
# Carregar dades Baix Ter
def loadDataBaixTer():
    baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)
    pasteralCabal = calcularAnomaly(baixTer, "F001242")
    colomers = calcularAnomaly(baixTer, "L17055-72-00002")
    torroellaMontegri = calcularAnomaly(baixTer, "L17199-72-00001")
    
    return pasteralCabal, colomers, torroellaMontegri

In [102]:
def selectTimespan(df, start_date, end_date):
    df = df.loc[(df['timestamp'] >= start_date) &  (df['timestamp'] <= end_date)]
    return df

In [103]:
# Carregar dades rius

alt1, alt2 = loadDataAltTer()
baix1, baix2, baix3 = loadDataBaixTer()


## Periode Temporal

In [104]:
#Seleccionem el periode temporal (en aquesta part el temporal Gloria)

start_date = datetime.datetime.strptime("20/09/2020", "%d/%m/%Y")
end_date = datetime.datetime.strptime("26/09/2020", "%d/%m/%Y")

alt1 = selectTimespan(alt1, start_date, end_date)
alt2 = selectTimespan(alt2, start_date, end_date)
baix1 = selectTimespan(baix1, start_date, end_date)
baix2 = selectTimespan(baix2, start_date, end_date)
baix3 = selectTimespan(baix3, start_date, end_date)

## Pearson

Ordre rius

- Alt Ter: Sant Joan de les Abadesses (L17167-72-00001) > Massies de Roda (L08116-72-00002)

- Baix Ter: Pasteral-Cabal (F001242) > Colomers (L17055-72-00002) > Torroella de Montgrí (L17199-72-00001)

In [106]:
def pearson_forward_lag(x, y, debug=False, n=40,m=40, p_ini=0):
    timeList = []
    rList = []
    pList = []
    offset = 0
    time = "00:00"
    timeStr = "%H:%M"
    time = datetime.datetime.strptime(time, timeStr)
    while offset < m:
        r, p = stats.pearsonr(x[p_ini:n], y[p_ini+offset:offset+n])
        if debug == True:
            print(f"Time Lag: +{time.strftime('%H:%M')}h , Pearson r: {r} and p-value: {p}")
        timeList.append(time)
        rList.append(r)
        pList.append(p)
        time = time + timedelta(minutes=30)
        offset += 1
        
    return timeList, rList, pList

In [107]:
def normalize_column(df, col):
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df

## Analisis Alt Ter (Sant Joan de les Abadesses > Massies de Roda)

In [108]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(alt1['data'], alt2['data'])

In [109]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig

## Analis Baix Ter (Pasteral-Cabal > Colomers)

In [111]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(baix1['data'], baix2['data'])


In [112]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig

## Analis Baix Ter (Colomers > Torroella de Montegri)

In [114]:
# Analisis Alt Ter
t, r, p = pearson_forward_lag(baix2['data'], baix3['data'])

In [115]:
# Fer un dataframe amb les dades aconseguides
d = {'r':r, 'p':p}
df = pd.DataFrame(data=d,index=t)
df.index = df.index.strftime('%H:%M')
fig = px.line(df, x=df.index, y=["r", "p"], markers=True)
fig